# [Titanic - Machine learning from Disaster](https://www.kaggle.com/competitions/titanic/)

- [My notebook](https://www.kaggle.com/code/param302/titanic-survival-competition-3)

## Importing required libraries

In [2]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

# Approach
1. Load data
2. Select useful features
3. Extract numerical and categorical columns
4. Make pipeline
5. Do Hyperparameter tuning with cross-validation
6. Check MAE
7. Find best parameters
8. Train model
9. Predicting `survived` on `test_data`
10. Submit predictions

## 1. Loading data

In [3]:
titanic_data = pd.read_csv("../input/titanic/train.csv", index_col="PassengerId")
test_data = pd.read_csv("../input/titanic/test.csv", index_col="PassengerId")

titanic_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2. Selecting useful features

In [4]:
useful_features = ["Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Ticket", "Fare", "Embarked"]
X = titanic_data[useful_features]
Y = titanic_data["Survived"]
test_data = test_data[useful_features]

## 3. Extracting numerical and categorical columns

In [5]:
num_cols = X.select_dtypes(exclude="object").columns
cat_cols = X.select_dtypes("object").columns
num_cols, cat_cols

(Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object'),
 Index(['Name', 'Sex', 'Ticket', 'Embarked'], dtype='object'))

## 4. Making pipeline

In [6]:
class CreatePipeline:
    """Create Pipeline
    methods:
        pipeline: Create Final Pipeline
        
        create_model: Create the provided model
        
        numerical_transformer: Transform numerical cols
        
        categorical_transformer: Transform categorical cols \
        OneHotEncoding / OrdinalEncoding
        
        data_preprocessor: Preprocess the data using ColumnTransformer     
        """
    
    def pipeline(self, *, preprocessor, model, verbose=False):
        """Creates pipeline
        params:
            preprocessor
            model
        """
        steps = [("preprocessor", preprocessor),
                 ("model", model)]
        return Pipeline(steps=steps, verbose=verbose)
    
    
    def numerical_transformer(self, *, strategy="mean", **params):
        """Transform numerical columns using `SimpleImputer`.
        params:
            strategy: "mean" | "median" | "most_frequent" | "constant"
            **params: extra keyword args for SimpleImputer"""
        
        transformer = SimpleImputer(strategy=strategy, **params)
        return transformer

    
    def categorical_transformer(self, *, 
                                imp_strategy="most_frequent", 
                                encoder_type="Ordinal", 
                                imp_params={}, encoder_params={}):
        """Transform categorical columns by making Pipeline
        `SimpleImputer` | `OneHotEncoder` | `OrdinalEncoder`.
        args:
            imp_strategy: strategy for imputer values can be
                "most_frequent" | "constant"
            encoder_type: encoder type,
                "Ordinal" | "OneHot"
        kwargs:
            imp_params: keyword args for `SimpleImputer`.
            encoder_params: keyword args for encoder.`
        """
        if not encoder_type in ("Ordinal", "OneHot"):
            raise ValueError(f"Inappropriate value for encoder_type passed: {encoder_type}\
            Takes one of 'Ordinal' | 'OneHot'.")
        
        encoder = OrdinalEncoder if encoder_type=="Ordinal" else OneHotEncoder
        transformer = Pipeline(steps=[
            ("imputer", SimpleImputer(strategy=imp_strategy, **imp_params)),
            (encoder_type, encoder(**encoder_params))
        ])
        return transformer
    
    
    def data_preprocessor(self, *, transformers):
        """Preprocess the data using `ColumnTransformer`.
        Pass extact list of transformers
        to be passed in `ColumnTransformer`.
        each tuple consist of: (transformer_name,
                                transformer,
                                list_of_columns)."""
        preprocessor = ColumnTransformer(transformers=transformers)
        return preprocessor
    
    
    def create_model(self, *, model, random_state=0, n_estimators=1000, **kwargs):
        """Creates the model.
        **kwargs: keyword args for model."""
        my_model = model(random_state=random_state, n_estimators=n_estimators, **kwargs)
        return my_model

In [7]:
cp = CreatePipeline()
num_transformer = cp.numerical_transformer()
cat_transformer = cp.categorical_transformer(encoder_type="OneHot", encoder_params={"handle_unknown": "ignore"})
preprocessor = cp.data_preprocessor(
                    transformers=[("num", num_transformer, num_cols),
                                  ("cat", cat_transformer, cat_cols)
                                 ])

## 5. Doing hyperparameter tuning with Cross-validation using `XGBClassifier`

In [8]:
n_estimators = [350, 500, 750]
max_depths = [5, 10, 20]
learning_rate = [0.05, 0.1]
maes = {}
i = 0
for n in n_estimators:
    for md in max_depths:
        for rate in learning_rate:
            i += 1
            model = cp.create_model(model=XGBClassifier, n_estimators=n, max_depth=md, learning_rate=rate)
            pipeline = cp.pipeline(preprocessor=preprocessor, model=model)
            scores = -1 * cross_val_score(pipeline, X, Y, cv=10, verbose=True,
                                    scoring="neg_mean_absolute_error")
            mae = scores.mean()
            maes[i] = [n, md, rate, mae]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   16.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   15.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   19.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   18.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   21.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   21.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

## 7. Checking MAE

In [9]:
for i in maes:
    n, md, rate, mae = maes[i]
    print(f"{i}.\tN_estimators: {n}\tmax_depth: {md}\tlearning_rate: {rate}\tMAE: {mae}")

1.	N_estimators: 350	max_depth: 5	learning_rate: 0.05	MAE: 0.1615730337078652
2.	N_estimators: 350	max_depth: 5	learning_rate: 0.1	MAE: 0.17952559300873905
3.	N_estimators: 350	max_depth: 10	learning_rate: 0.05	MAE: 0.19300873907615482
4.	N_estimators: 350	max_depth: 10	learning_rate: 0.1	MAE: 0.18851435705368288
5.	N_estimators: 350	max_depth: 20	learning_rate: 0.05	MAE: 0.18403245942571786
6.	N_estimators: 350	max_depth: 20	learning_rate: 0.1	MAE: 0.19076154806491885
7.	N_estimators: 500	max_depth: 5	learning_rate: 0.05	MAE: 0.17054931335830212
8.	N_estimators: 500	max_depth: 5	learning_rate: 0.1	MAE: 0.17726591760299626
9.	N_estimators: 500	max_depth: 10	learning_rate: 0.05	MAE: 0.19300873907615482
10.	N_estimators: 500	max_depth: 10	learning_rate: 0.1	MAE: 0.18963795255930085
11.	N_estimators: 500	max_depth: 20	learning_rate: 0.05	MAE: 0.18739076154806492
12.	N_estimators: 500	max_depth: 20	learning_rate: 0.1	MAE: 0.19413233458177276
13.	N_estimators: 750	max_depth: 5	learning_rate

## 8. Best parameters
#### `n_estimators: 350`
#### `max_depth: 5`
#### `learning_rate: 0.05`
#### `MAE: 0.1615730337078652`

In [10]:
best_n_estimators = 350
best_max_depth = 5
best_rate = 0.05

## 9. Training `XGBClassifier` model with best parameters

In [11]:
model = cp.create_model(model=XGBClassifier, n_estimators=best_n_estimators, learning_rate=best_rate, max_depth=best_max_depth)
pipeline = cp.pipeline(preprocessor=preprocessor, model=model)
pipeline.fit(X, Y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Name', 'Sex', 'Ticket', 'Embarked'], dtype='object'))])),
                ('model',
                 XGB...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                    

## 10. Predicting `Survived` on `test_data`

In [12]:
test_preds = pipeline.predict(test_data)
test_preds

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## 11. Submitting predictions

In [13]:
output = pd.DataFrame({"PassengerId": test_data.index, "Survived": test_preds})
output.to_csv("./data/submission_3.csv", index=False)